<a href="https://colab.research.google.com/github/mejriimariemm-12/veristream-x/blob/main/factscheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 72.3 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re

DATA_DIR = "medical_sources"
os.makedirs(DATA_DIR, exist_ok=True)

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[.*?\]', '', text)
    return text.strip()

def scrape_and_clean(url, name):
    print(f"Scraping: {url}")

    r = requests.get(url, timeout=30)
    soup = BeautifulSoup(r.text, "html.parser")

    # Enlever les scripts et menus
    for tag in soup(["script", "style", "header", "footer", "nav", "aside"]):
        tag.decompose()

    text = soup.get_text(separator=" ")
    text = clean_text(text)

    # Sauvegarde
    file_path = os.path.join(DATA_DIR, f"{name}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"✔ Saved cleaned text → {file_path}")
    return file_path


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
import time
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Définir un seul répertoire pour toutes les opérations
DATA_DIR = "medical_sources"
os.makedirs(DATA_DIR, exist_ok=True)

def clean_text(text):
    """Nettoie le texte extrait"""
    text = re.sub(r'\s+', ' ', text)  # Remplace les espaces multiples
    text = re.sub(r'\[.*?\]', '', text)  # Supprime les crochets
    text = re.sub(r'\n+', ' ', text)  # Supprime les retours à ligne multiples
    # Supprimer les URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Supprimer les emails
    text = re.sub(r'\S+@\S+', '', text)
    return text.strip()

def scrape_and_clean(url, name, max_retries=2):
    """Scrape, nettoie et sauvegarde le contenu d'une URL avec retry"""
    print(f"Scraping: {url}")

    # Headers pour éviter le blocage
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

    for attempt in range(max_retries):
        try:
            r = requests.get(url, timeout=45, headers=headers)
            r.raise_for_status()
            break
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                print(f"⚠ Timeout, nouvelle tentative {attempt + 2}/{max_retries}...")
                time.sleep(2)
                continue
            else:
                print(f"❌ Timeout après {max_retries} tentatives: {url}")
                return None
        except requests.RequestException as e:
            print(f"❌ Erreur lors du scraping de {url}: {e}")
            return None

    # Vérifier l'encodage
    if r.encoding != 'utf-8':
        r.encoding = r.apparent_encoding or 'utf-8'

    soup = BeautifulSoup(r.text, "html.parser")

    # Supprimer les éléments inutiles de manière plus agressive
    for tag in soup(["script", "style", "header", "footer", "nav", "aside",
                     "iframe", "form", "button", "input", "select", "textarea",
                     "noscript", "meta", "link", "svg", "path"]):
        tag.decompose()

    # Supprimer les divs avec des classes courantes de menus/sidebars
    for div in soup.find_all("div", class_=re.compile(r"(menu|sidebar|navigation|footer|header|banner|popup|modal|ads|advertisement)", re.I)):
        div.decompose()

    # Essayer d'extraire le contenu principal
    main_content = None
    for selector in ['main', 'article', 'div.content', 'div.main-content',
                     'div.post-content', 'div.entry-content', '#content', '.content']:
        main_content = soup.select_one(selector)
        if main_content:
            break

    if main_content:
        text = main_content.get_text(separator=" ", strip=True)
    else:
        # Essayer body
        body = soup.find('body')
        if body:
            text = body.get_text(separator=" ", strip=True)
        else:
            text = soup.get_text(separator=" ", strip=True)

    text = clean_text(text)

    # Vérifier que le texte n'est pas trop court (page vide)
    if len(text) < 300:
        print(f"⚠ Attention: Le texte extrait de {name} est court ({len(text)} caractères)")
        # Essayer une extraction plus large
        text = soup.get_text(separator=" ", strip=True)
        text = clean_text(text)

    # Sauvegarder dans le répertoire défini
    file_path = os.path.join(DATA_DIR, f"{name}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(f"URL Source: {url}\n")
        f.write(f"Date de scraping: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 50 + "\n\n")
        f.write(text)

    print(f"✔ Saved cleaned text ({len(text):,} caractères) → {file_path}")
    return file_path

# 2. Liste des sources à scraper (URLs vérifiées)
sources = {
    "OMS_covid_vaccins": "https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-(covid-19)-vaccines",
    "OMS_vaccination_basics": "https://www.who.int/news-room/fact-sheets/detail/immunization",  # Alternative
    "CDC_covid_vaccines": "https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines.html",
    "CDC_vaccine_safety": "https://www.cdc.gov/vaccinesafety/",  # Alternative
    "HAS_vaccins_covid": "https://www.has-sante.fr/jcms/p_3228228/fr/vaccination-contre-la-covid-19",
    "SPF_vaccination": "https://solidarites-sante.gouv.fr/grands-dossiers/vaccination-covid-19/",  # Santé Publique France
    "EMA_vaccines": "https://www.ema.europa.eu/en/human-regulatory/overview/public-health-threats/coronavirus-disease-covid-19/treatments-vaccines/vaccines-covid-19",  # Agence Européenne
    "NIH_covid_treatment": "https://www.covid19treatmentguidelines.nih.gov/overview/",  # NIH américain
}

# 3. Scraper toutes les sources
print("=" * 60)
print("DÉBUT DU SCRAPING DES SOURCES MÉDICALES")
print("=" * 60)

files = []
success_count = 0

for name, url in sources.items():
    file_path = scrape_and_clean(url, name)
    if file_path:
        files.append(file_path)
        success_count += 1
    print("-" * 40)
    time.sleep(1)  # Pause pour éviter le blocage

print(f"\n📊 RÉSUMÉ DU SCRAPING:")
print(f"   Sources totales: {len(sources)}")
print(f"   Succès: {success_count}")
print(f"   Échecs: {len(sources) - success_count}")
print(f"   Fichiers sauvegardés dans '{DATA_DIR}/'")

# 4. Charger et diviser les documents
print("\n" + "=" * 60)
print("CHARGEMENT ET DIVISION DES DOCUMENTS")
print("=" * 60)

def load_all_documents(directory):
    """Charge tous les documents texte d'un répertoire"""
    docs = []
    txt_files = [f for f in os.listdir(directory) if f.endswith(".txt")]

    if not txt_files:
        print(f"❌ Aucun fichier .txt trouvé dans '{directory}'")
        return docs

    print(f"📁 Fichiers trouvés: {len(txt_files)}")

    for file in sorted(txt_files):
        try:
            file_path = os.path.join(directory, file)
            loader = TextLoader(file_path, encoding="utf-8")
            loaded_docs = loader.load()

            # Ajouter des métadonnées supplémentaires
            for doc in loaded_docs:
                doc.metadata["filename"] = file
                doc.metadata["source_name"] = file.replace(".txt", "")

            docs.extend(loaded_docs)
            print(f"✓ Chargé: {file} ({len(loaded_docs[0].page_content):,} caractères)")
        except Exception as e:
            print(f"✗ Erreur avec {file}: {e}")
    return docs

# Utiliser le même répertoire DATA_DIR
if os.path.exists(DATA_DIR) and os.listdir(DATA_DIR):
    # Charger les documents
    docs = load_all_documents(DATA_DIR)

    if docs:
        print(f"\n📄 Documents chargés: {len(docs)}")

        # Configurer le splitter avec des paramètres optimisés
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,  # Augmenté pour plus de contexte
            chunk_overlap=150,
            length_function=len,
            separators=["\n\n", "\n", ". ", "? ", "! ", "; ", ", ", " ", ""],
            keep_separator=True
        )

        # Diviser en chunks
        chunks = splitter.split_documents(docs)
        print(f"✅ Nombre de chunks créés : {len(chunks)}")

        # Afficher des statistiques
        total_chars = sum(len(chunk.page_content) for chunk in chunks)
        avg_chunk_size = total_chars / len(chunks) if chunks else 0

        print(f"\n📊 STATISTIQUES DES CHUNKS:")
        print(f"   Taille moyenne: {avg_chunk_size:.0f} caractères")
        print(f"   Total de texte: {total_chars:,} caractères")
        print(f"   Min: {min(len(c.page_content) for c in chunks) if chunks else 0} caractères")
        print(f"   Max: {max(len(c.page_content) for c in chunks) if chunks else 0} caractères")

        # Filtrer les chunks trop courts
        min_chunk_size = 200
        filtered_chunks = [c for c in chunks if len(c.page_content) >= min_chunk_size]

        if len(filtered_chunks) < len(chunks):
            print(f"\n⚠ Filtrage: {len(chunks) - len(filtered_chunks)} chunks de moins de {min_chunk_size} caractères retirés")
            chunks = filtered_chunks
            print(f"   Chunks restants: {len(chunks)}")

        # Afficher des échantillons
        if chunks:
            print("\n" + "=" * 60)
            print("ÉCHANTILLONS DE CHUNKS:")
            print("=" * 60)

            for i in range(min(3, len(chunks))):
                print(f"\n🎯 CHUNK {i+1} (Source: {chunks[i].metadata.get('filename', 'Inconnu')}):")
                print("-" * 40)
                sample = chunks[i].page_content[:400]
                print(f"{sample}...")
                print(f"Taille: {len(chunks[i].page_content)} caractères")
                print("-" * 40)

            # Sauvegarder les chunks pour un usage futur
            chunks_dir = os.path.join(DATA_DIR, "chunks")
            os.makedirs(chunks_dir, exist_ok=True)

            # Sauvegarder un fichier avec tous les chunks
            chunks_file = os.path.join(DATA_DIR, "all_chunks_metadata.txt")
            with open(chunks_file, "w", encoding="utf-8") as f:
                f.write("MÉTADONNÉES DES CHUNKS\n")
                f.write("=" * 60 + "\n\n")

                for i, chunk in enumerate(chunks):
                    f.write(f"\nCHUNK {i}:\n")
                    f.write(f"Source: {chunk.metadata.get('filename', 'Unknown')}\n")
                    f.write(f"Taille: {len(chunk.page_content)} caractères\n")
                    f.write("-" * 40 + "\n")
                    # Premières 200 caractères
                    f.write(chunk.page_content[:200] + "...\n")

            print(f"\n💾 Métadonnées sauvegardées: {chunks_file}")

            # Optionnel: sauvegarder chaque chunk individuellement
            save_individual = input("\n💾 Voulez-vous sauvegarder chaque chunk individuellement? (o/n): ")
            if save_individual.lower() == 'o':
                for i, chunk in enumerate(chunks):
                    chunk_file = os.path.join(chunks_dir, f"chunk_{i:03d}.txt")
                    with open(chunk_file, "w", encoding="utf-8") as f:
                        f.write(f"Chunk {i}\n")
                        f.write(f"Source: {chunk.metadata.get('filename', 'Unknown')}\n")
                        f.write(f"URL: {chunk.metadata.get('source', 'Unknown')}\n")
                        f.write("-" * 50 + "\n")
                        f.write(chunk.page_content)
                print(f"✅ {len(chunks)} chunks sauvegardés dans: {chunks_dir}/")

            # Préparer pour RAG
            print("\n" + "=" * 60)
            print("PRÉPARATION POUR RAG:")
            print("=" * 60)
            print("Les chunks sont prêts pour être utilisés avec:")
            print("1. Embeddings (FAISS, Chroma, Pinecone)")
            print("2. Modèles LLM (GPT, Llama, Mistral)")
            print("3. Systèmes de question-réponse")
            print(f"\nVous avez {len(chunks)} chunks de qualité pour votre base de connaissances.")

    else:
        print("❌ Aucun document chargé.")
else:
    print(f"❌ Le répertoire '{DATA_DIR}' est vide ou n'existe pas.")
    print("Veuillez d'abord exécuter la partie scraping.")

# 5. Suggestions pour améliorer
print("\n" + "=" * 60)
print("SUGGESTIONS POUR AMÉLIORER:")
print("=" * 60)
print("1. Ajouter plus de sources francophones:")
print("   - Ministère de la Santé")
print("   - Institut Pasteur")
print("   - Revue médicale")
print("\n2. Pour les sites bloquants, utiliser:")
print("   - Selenium pour JavaScript")
print("   - APIs officielles si disponibles")
print("   - Retards plus longs entre les requêtes")
print("\n3. Qualité des chunks:")
print("   - Ajuster chunk_size selon votre use case")
print("   - Ajouter du preprocessing supplémentaire")

DÉBUT DU SCRAPING DES SOURCES MÉDICALES
Scraping: https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-(covid-19)-vaccines
✔ Saved cleaned text (12,517 caractères) → medical_sources/OMS_covid_vaccins.txt
----------------------------------------
Scraping: https://www.who.int/news-room/fact-sheets/detail/immunization
⚠ Attention: Le texte extrait de OMS_vaccination_basics est court (34 caractères)
✔ Saved cleaned text (62 caractères) → medical_sources/OMS_vaccination_basics.txt
----------------------------------------
Scraping: https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines.html
❌ Erreur lors du scraping de https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines.html: 404 Client Error: Not Found for url: https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines.html
----------------------------------------
Scraping: https://www.cdc.gov/vaccinesafety/
❌ Erreur lors du scraping de https://www.cdc.gov/vaccinesafety/: 4

In [ ]:
# ============================================================
# RAG MÉDICAL COMPLET ET AUTONOME
# ============================================================

print("🚀 RAG MÉDICAL COMPLET - CRÉATION DE A à Z")
print("="*60)

# 1. INSTALLATION
!pip install -q sentence-transformers faiss-cpu

# 2. IMPORTS
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import json

print("✅ Dépendances installées")

# 3. CRÉATION DES DONNÉES MÉDICALES
print("\n📝 Création de la base de connaissances médicale...")

medical_documents = [
    {
        "title": "Antibiotiques et infections",
        "content": "Les antibiotiques sont des médicaments utilisés pour traiter les infections bactériennes. Ils NE sont PAS efficaces contre les infections virales comme le rhume ou la grippe. Utilisation uniquement sur prescription médicale. Respecter la dose et la durée du traitement. La résistance aux antibiotiques est un problème mondial de santé publique."
    },
    {
        "title": "Vaccins COVID-19",
        "content": "Les vaccins contre la COVID-19 ont été rigoureusement testés. Ils réduisent le risque d'hospitalisation de 90%. Les effets secondaires sont généralement légers: douleur au point d'injection, fatigue, maux de tête. La vaccination est recommandée pour les personnes à risque."
    },
    {
        "title": "Prévention des infections",
        "content": "Pour prévenir les infections: se laver les mains régulièrement, tousser dans son coude, porter un masque en cas de symptômes, aérer les pièces. La vaccination annuelle contre la grippe est recommandée."
    },
    {
        "title": "Symptômes COVID-19",
        "content": "Symptômes principaux: fièvre, toux sèche, fatigue. Autres symptômes: perte du goût/odorat, maux de gorge, diarrhée. Symptômes graves: difficultés respiratoires, douleurs thoraciques. Consulter un médecin en cas de symptômes graves."
    },
    {
        "title": "Renforcement immunitaire",
        "content": "Pour renforcer le système immunitaire: alimentation équilibrée riche en fruits et légumes, sommeil suffisant (7-9h), activité physique régulière, gestion du stress. Éviter le tabac et modérer l'alcool."
    },
    {
        "title": "Traitement de la fièvre",
        "content": "La fièvre est une défense naturelle. Traitement: paracétamol selon posologie, boire beaucoup d'eau, ne pas trop couvrir. Consulter si fièvre > 40°C, durée > 3 jours, ou chez un enfant de moins de 3 mois."
    }
]

print(f"✅ {len(medical_documents)} documents médicaux créés")

# 4. CRÉATION DES CHUNKS
print("\n✂️ Création des chunks...")

chunks = []
for doc_id, doc in enumerate(medical_documents):
    # Créer 2 chunks par document
    content = doc["content"]
    sentences = content.split('. ')

    # Premier chunk (premières phrases)
    chunk1 = '. '.join(sentences[:len(sentences)//2]) + '.'
    chunks.append({
        "id": len(chunks),
        "text": chunk1,
        "title": doc["title"],
        "document_id": doc_id
    })

    # Deuxième chunk (dernières phrases)
    chunk2 = '. '.join(sentences[len(sentences)//2:]) + '.'
    chunks.append({
        "id": len(chunks),
        "text": chunk2,
        "title": doc["title"],
        "document_id": doc_id
    })

print(f"✅ {len(chunks)} chunks créés")

# Afficher quelques chunks
print("\n📋 Exemples de chunks:")
for i in range(min(3, len(chunks))):
    print(f"\nChunk {i+1} - {chunks[i]['title']}:")
    print(f"  {chunks[i]['text'][:100]}...")

# 5. EMBEDDINGS
print("\n🔤 Création des embeddings...")

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
texts = [chunk["text"] for chunk in chunks]
embeddings = model.encode(texts, show_progress_bar=True)

print(f"✅ Embeddings: {embeddings.shape}")

# 6. INDEX FAISS
print("\n🔍 Construction de l'index FAISS...")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

print(f"✅ Index: {index.ntotal} documents, dimension {dimension}")

# 7. SYSTÈME RAG OPTIMISÉ
print("\n🤖 Création du système RAG optimisé...")

class MedicalRAGSystem:
    """Système RAG médical optimisé"""

    def __init__(self, index, chunks, embedding_model):
        self.index = index
        self.chunks = chunks
        self.model = embedding_model

    def clean_text(self, text):
        """Nettoie le texte"""
        return ' '.join(text.split())

    def search_documents(self, query, k=3):
        """Recherche les documents pertinents"""
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(query_embedding.astype('float32'), k)

        results = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.chunks):
                similarity = 1 / (1 + distances[0][i])
                results.append({
                    "chunk": self.chunks[idx],
                    "similarity": float(similarity)
                })

        results.sort(key=lambda x: x["similarity"], reverse=True)
        return results

    def classify_question(self, question):
        """Détermine le type de question"""
        question = question.lower()

        if any(word in question for word in ["antibiotique", "bactérie", "virus"]):
            return "antibiotiques"
        elif any(word in question for word in ["vaccin", "covid", "vaccination"]):
            return "vaccins"
        elif any(word in question for word in ["prévenir", "éviter", "grippe"]):
            return "prévention"
        elif any(word in question for word in ["symptôme", "fièvre", "toux"]):
            return "symptômes"
        elif any(word in question for word in ["immunitaire", "défense", "renforcer"]):
            return "immunité"
        else:
            return "général"

    def generate_answer(self, question):
        """Génère une réponse concise"""
        print(f"\n🔍 Question: {question}")

        # Rechercher
        results = self.search_documents(question)

        if not results:
            return self.get_default_answer(question)

        print(f"📊 {len(results)} source(s) trouvée(s)")

        # Type de question
        q_type = self.classify_question(question)

        # Extraire les informations pertinentes
        main_info = ""
        additional_info = ""

        if results:
            main_chunk = results[0]["chunk"]
            main_text = self.clean_text(main_chunk["text"])
            main_info = main_text[:250] + "..." if len(main_text) > 250 else main_text

            if len(results) > 1:
                second_chunk = results[1]["chunk"]
                second_text = self.clean_text(second_chunk["text"])
                additional_info = second_text[:150] + "..." if len(second_text) > 150 else second_text

        # Construire la réponse
        response = self.format_response(q_type, main_info, additional_info)

        # Sources utilisées
        sources = []
        for result in results[:2]:
            sources.append({
                "title": result["chunk"]["title"],
                "similarity": result["similarity"]
            })

        # Confiance
        confidence = sum(r["similarity"] for r in results[:2]) / min(2, len(results))

        return {
            "answer": response,
            "sources": sources,
            "confidence": confidence,
            "type": q_type
        }

    def format_response(self, q_type, main_info, additional_info):
        """Formate la réponse"""
        # Titres
        titles = {
            "antibiotiques": "💊 INFORMATION SUR LES ANTIBIOTIQUES",
            "vaccins": "💉 INFORMATION SUR LES VACCINS",
            "prévention": "🛡️ PRÉVENTION DES INFECTIONS",
            "symptômes": "🤒 SYMPTÔMES ET SIGNES",
            "immunité": "🌿 SYSTÈME IMMUNITAIRE",
            "général": "📋 INFORMATION MÉDICALE"
        }

        title = titles.get(q_type, "📋 INFORMATION MÉDICALE")

        # Construire la réponse
        response = f"{title}\n{'='*40}\n\n"
        response += f"{main_info}\n\n"

        if additional_info:
            response += f"💡 Également: {additional_info}\n\n"

        # Recommandation finale
        response += self.get_recommendation(q_type)

        return response

    def get_recommendation(self, q_type):
        """Retourne une recommandation appropriée"""
        recommendations = {
            "antibiotiques": "⚠️ Consultez toujours un médecin avant de prendre des antibiotiques.",
            "vaccins": "✅ Parlez à votre médecin des vaccins recommandés pour vous.",
            "prévention": "🛡️ Une bonne hygiène est la meilleure prévention.",
            "symptômes": "🏥 Consultez un médecin en cas de symptômes graves.",
            "immunité": "🌿 Un mode de vie sain soutient votre système immunitaire.",
            "général": "ℹ️ Pour des conseils personnalisés, consultez un professionnel de santé."
        }
        return recommendations.get(q_type, "Consultez un professionnel de santé.")

    def get_default_answer(self, question):
        """Réponse par défaut"""
        q_type = self.classify_question(question)

        default_answers = {
            "antibiotiques": "💊 INFORMATION SUR LES ANTIBIOTIQUES\nLes antibiotiques traitent les infections bactériennes, pas virales. Prescription médicale obligatoire.",
            "vaccins": "💉 INFORMATION SUR LES VACCINS\nLes vaccins sont des outils de prévention essentiels. Discutez avec votre médecin.",
            "prévention": "🛡️ PRÉVENTION DES INFECTIONS\nLavage des mains, vaccination et bonne hygiène réduisent les risques.",
            "symptômes": "🤒 SYMPTÔMES ET SIGNES\nLes symptômes varient selon les infections. Consultez en cas de doute.",
            "immunité": "🌿 SYSTÈME IMMUNITAIRE\nAlimentation équilibrée, sommeil et activité physique renforcent les défenses.",
            "général": "📋 INFORMATION MÉDICALE\nPour des informations précises, posez une question spécifique."
        }

        return {
            "answer": default_answers.get(q_type, default_answers["général"]),
            "sources": [],
            "confidence": 0.0,
            "type": q_type
        }

# 8. INITIALISATION DU SYSTÈME
print("🔄 Initialisation du système...")
rag_system = MedicalRAGSystem(index, chunks, model)
print("✅ Système RAG prêt!")

# ============================================================
# TESTS AUTOMATIQUES
# ============================================================
print("\n" + "="*60)
print("🧪 TESTS AUTOMATIQUES")
print("="*60)

test_questions = [
    "Les antibiotiques soignent-ils la grippe?",
    "Le vaccin COVID est-il efficace?",
    "Comment éviter d'attraper la grippe?",
    "Quels sont les symptômes du COVID?",
    "Comment renforcer mon système immunitaire?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*50}")
    print(f"TEST {i}: {question}")

    result = rag_system.generate_answer(question)

    print(f"\n{result['answer']}")

    if result['sources']:
        print(f"\n📚 Sources:")
        for j, source in enumerate(result['sources'], 1):
            print(f"  {j}. {source['title']} ({source['similarity']:.0%} pertinente)")

    print(f"\n🎯 Confiance: {result['confidence']:.0%}")

# ============================================================
# INTERFACE UTILISATEUR
# ============================================================
print("\n" + "="*60)
print("💬 INTERACTIF - POSEZ VOS QUESTIONS")
print("="*60)
print("Tapez vos questions médicales (ou 'stop' pour arrêter)")

question_count = 1
while True:
    try:
        user_question = input(f"\n❓ Question {question_count}: ").strip()

        if user_question.lower() in ['stop', 'exit', 'quit', 'fin', '']:
            print("\n👋 Fin de la session. Prenez soin de votre santé!")
            break

        if not user_question:
            continue

        # Obtenir la réponse
        result = rag_system.generate_answer(user_question)

        # Afficher
        print(f"\n{'='*60}")
        print("🤖 RÉPONSE:")
        print(f"{result['answer']}")
        print(f"{'='*60}")

        if result['sources']:
            print(f"\n📖 Basé sur {len(result['sources'])} source(s) médicale(s)")

        print(f"\n🔍 Score de pertinence: {result['confidence']:.0%}")

        question_count += 1

    except KeyboardInterrupt:
        print("\n\n👋 Session interrompue")
        break
    except Exception as e:
        print(f"\n⚠️ Erreur: {e}")
        continue

# ============================================================
# RÉSUMÉ FINAL
# ============================================================
print("\n" + "="*60)
print("📊 RÉSUMÉ DU SYSTÈME")
print("="*60)

print(f"""
✅ SYSTÈME RAG MÉDICAL OPÉRATIONNEL

📚 BASE DE CONNAISSANCES:
  • Documents: {len(medical_documents)} sujets médicaux
  • Chunks: {len(chunks)} extraits d'information
  • Sujets couverts:
     1. Antibiotiques et infections
     2. Vaccins COVID-19
     3. Prévention des infections
     4. Symptômes COVID-19
     5. Renforcement immunitaire
     6. Traitement de la fièvre

🧠 TECHNIQUE:
  • Modèle: paraphrase-multilingual-MiniLM-L12-v2
  • Embeddings: {dimension} dimensions
  • Index: FAISS avec {index.ntotal} vecteurs
  • Recherche: similarité cosinus

🎯 CAPACITÉS:
  • Réponses concises et ciblées
  • Classification automatique des questions
  • Sources citées avec score de pertinence
  • Recommandations adaptées
  • Interface interactive simple

⚠️ LIMITATIONS:
  • Base limitée à {len(medical_documents)} sujets
  • Ne remplace pas un avis médical
  • Informations générales uniquement
""")

print("\n" + "="*60)
print("🎉 RAG MÉDICAL FONCTIONNEL!")
print("="*60)
print("Le système est prêt à répondre à vos questions médicales.")

🚀 RAG MÉDICAL COMPLET - CRÉATION DE A à Z
✅ Dépendances installées

📝 Création de la base de connaissances médicale...
✅ 6 documents médicaux créés

✂️ Création des chunks...
✅ 12 chunks créés

📋 Exemples de chunks:

Chunk 1 - Antibiotiques et infections:
  Les antibiotiques sont des médicaments utilisés pour traiter les infections bactériennes. Ils NE son...

Chunk 2 - Antibiotiques et infections:
  Utilisation uniquement sur prescription médicale. Respecter la dose et la durée du traitement. La ré...

Chunk 3 - Vaccins COVID-19:
  Les vaccins contre la COVID-19 ont été rigoureusement testés. Ils réduisent le risque d'hospitalisat...

🔤 Création des embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings: (12, 384)

🔍 Construction de l'index FAISS...
✅ Index: 12 documents, dimension 384

🤖 Création du système RAG optimisé...
🔄 Initialisation du système...
✅ Système RAG prêt!

🧪 TESTS AUTOMATIQUES

TEST 1: Les antibiotiques soignent-ils la grippe?

🔍 Question: Les antibiotiques soignent-ils la grippe?
📊 3 source(s) trouvée(s)

💊 INFORMATION SUR LES ANTIBIOTIQUES

Les antibiotiques sont des médicaments utilisés pour traiter les infections bactériennes. Ils NE sont PAS efficaces contre les infections virales comme le rhume ou la grippe.

💡 Également: La vaccination annuelle contre la grippe est recommandée..

⚠️ Consultez toujours un médecin avant de prendre des antibiotiques.

📚 Sources:
  1. Antibiotiques et infections (12% pertinente)
  2. Prévention des infections (7% pertinente)

🎯 Confiance: 10%

TEST 2: Le vaccin COVID est-il efficace?

🔍 Question: Le vaccin COVID est-il efficace?
📊 3 source(s) trouvée(s)

💉 INFORMATION SUR LES VACCINS

Les vaccins contre la COVID-19 o

In [ ]:
# ============================================================
# SAUVEGARDE DU SYSTÈME RAG POUR RÉUTILISATION
# ============================================================

print("💾 SAUVEGARDE DU SYSTÈME RAG")
print("="*60)

import pickle
import json
from datetime import datetime
import os

# 1. CRÉER UN DOSSIER DE SAUVEGARDE
save_dir = "medical_rag_saved"
os.makedirs(save_dir, exist_ok=True)

print(f"📁 Dossier de sauvegarde: {save_dir}")

# 2. SAUVEGARDER LES COMPOSANTS
print("\n💾 Sauvegarde des composants...")

# Sauvegarder l'index FAISS
faiss.write_index(index, f"{save_dir}/medical_index.bin")
print("✅ Index FAISS sauvegardé")

# Sauvegarder les embeddings
np.save(f"{save_dir}/embeddings.npy", embeddings)
print("✅ Embeddings sauvegardés")

# Sauvegarder les chunks
with open(f"{save_dir}/chunks.json", 'w', encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
print("✅ Chunks sauvegardés")

# Sauvegarder les documents
with open(f"{save_dir}/documents.json", 'w', encoding='utf-8') as f:
    json.dump(medical_documents, f, ensure_ascii=False, indent=2)
print("✅ Documents médicaux sauvegardés")

# Sauvegarder le système complet
system_data = {
    "dimension": dimension,
    "chunks_count": len(chunks),
    "documents_count": len(medical_documents),
    "model_name": "paraphrase-multilingual-MiniLM-L12-v2",
    "creation_date": datetime.now().isoformat(),
    "version": "1.0"
}

with open(f"{save_dir}/system_info.json", 'w', encoding='utf-8') as f:
    json.dump(system_data, f, indent=2)
print("✅ Informations système sauvegardées")

# Sauvegarder la classe MedicalRAGSystem pour réutilisation
# Créer un fichier Python séparé avec la classe
class_file_content = '''
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import json

class MedicalRAGSystem:
    """Système RAG médical optimisé"""

    def __init__(self, index, chunks, embedding_model):
        self.index = index
        self.chunks = chunks
        self.model = embedding_model

    def clean_text(self, text):
        """Nettoie le texte"""
        return ' '.join(text.split())

    def search_documents(self, query, k=3):
        """Recherche les documents pertinents"""
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(query_embedding.astype('float32'), k)

        results = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.chunks):
                similarity = 1 / (1 + distances[0][i])
                results.append({
                    "chunk": self.chunks[idx],
                    "similarity": float(similarity)
                })

        results.sort(key=lambda x: x["similarity"], reverse=True)
        return results

    def classify_question(self, question):
        """Détermine le type de question"""
        question = question.lower()

        if any(word in question for word in ["antibiotique", "bactérie", "virus"]):
            return "antibiotiques"
        elif any(word in question for word in ["vaccin", "covid", "vaccination"]):
            return "vaccins"
        elif any(word in question for word in ["prévenir", "éviter", "grippe"]):
            return "prévention"
        elif any(word in question for word in ["symptôme", "fièvre", "toux"]):
            return "symptômes"
        elif any(word in question for word in ["immunitaire", "défense", "renforcer"]):
            return "immunité"
        else:
            return "général"

    def generate_answer(self, question):
        """Génère une réponse concise"""
        print(f"🔍 Question: {question}")

        # Rechercher
        results = self.search_documents(question)

        if not results:
            return self.get_default_answer(question)

        print(f"📊 {len(results)} source(s) trouvée(s)")

        # Type de question
        q_type = self.classify_question(question)

        # Extraire les informations pertinentes
        main_info = ""
        additional_info = ""

        if results:
            main_chunk = results[0]["chunk"]
            main_text = self.clean_text(main_chunk["text"])
            main_info = main_text[:250] + "..." if len(main_text) > 250 else main_text

            if len(results) > 1:
                second_chunk = results[1]["chunk"]
                second_text = self.clean_text(second_chunk["text"])
                additional_info = second_text[:150] + "..." if len(second_text) > 150 else second_text

        # Construire la réponse
        response = self.format_response(q_type, main_info, additional_info)

        # Sources utilisées
        sources = []
        for result in results[:2]:
            sources.append({
                "title": result["chunk"]["title"],
                "similarity": result["similarity"]
            })

        # Confiance
        confidence = sum(r["similarity"] for r in results[:2]) / min(2, len(results))

        return {
            "answer": response,
            "sources": sources,
            "confidence": confidence,
            "type": q_type
        }

    def format_response(self, q_type, main_info, additional_info):
        """Formate la réponse"""
        # Titres
        titles = {
            "antibiotiques": "💊 INFORMATION SUR LES ANTIBIOTIQUES",
            "vaccins": "💉 INFORMATION SUR LES VACCINS",
            "prévention": "🛡️ PRÉVENTION DES INFECTIONS",
            "symptômes": "🤒 SYMPTÔMES ET SIGNES",
            "immunité": "🌿 SYSTÈME IMMUNITAIRE",
            "général": "📋 INFORMATION MÉDICALE"
        }

        title = titles.get(q_type, "📋 INFORMATION MÉDICALE")

        # Construire la réponse
        response = f"{title}\\n{'='*40}\\n\\n"
        response += f"{main_info}\\n\\n"

        if additional_info:
            response += f"💡 Également: {additional_info}\\n\\n"

        # Recommandation finale
        response += self.get_recommendation(q_type)

        return response

    def get_recommendation(self, q_type):
        """Retourne une recommandation appropriée"""
        recommendations = {
            "antibiotiques": "⚠️ Consultez toujours un médecin avant de prendre des antibiotiques.",
            "vaccins": "✅ Parlez à votre médecin des vaccins recommandés pour vous.",
            "prévention": "🛡️ Une bonne hygiène est la meilleure prévention.",
            "symptômes": "🏥 Consultez un médecin en cas de symptômes graves.",
            "immunité": "🌿 Un mode de vie sain soutient votre système immunitaire.",
            "général": "ℹ️ Pour des conseils personnalisés, consultez un professionnel de santé."
        }
        return recommendations.get(q_type, "Consultez un professionnel de santé.")

    def get_default_answer(self, question):
        """Réponse par défaut"""
        q_type = self.classify_question(question)

        default_answers = {
            "antibiotiques": "💊 INFORMATION SUR LES ANTIBIOTIQUES\\nLes antibiotiques traitent les infections bactériennes, pas virales. Prescription médicale obligatoire.",
            "vaccins": "💉 INFORMATION SUR LES VACCINS\\nLes vaccins sont des outils de prévention essentiels. Discutez avec votre médecin.",
            "prévention": "🛡️ PRÉVENTION DES INFECTIONS\\nLavage des mains, vaccination et bonne hygiène réduisent les risques.",
            "symptômes": "🤒 SYMPTÔMES ET SIGNES\\nLes symptômes varient selon les infections. Consultez en cas de doute.",
            "immunité": "🌿 SYSTÈME IMMUNITAIRE\\nAlimentation équilibrée, sommeil et activité physique renforcent les défenses.",
            "général": "📋 INFORMATION MÉDICALE\\nPour des informations précises, posez une question spécifique."
        }

        return {
            "answer": default_answers.get(q_type, default_answers["général"]),
            "sources": [],
            "confidence": 0.0,
            "type": q_type
        }
'''

with open(f"{save_dir}/medical_rag_class.py", 'w', encoding='utf-8') as f:
    f.write(class_file_content)
print("✅ Classe MedicalRAGSystem sauvegardée")

# 3. CRÉER UN FICHIER DE CHARGEMENT POUR LE PROCHAIN NOTEBOOK
loader_file_content = '''
# ============================================================
# CHARGEMENT DU SYSTÈME RAG MÉDICAL SAUVEGARDÉ
# ============================================================

print("📂 CHARGEMENT DU SYSTÈME RAG MÉDICAL")
print("="*60)

# 1. INSTALLATION (si nécessaire)
try:
    import faiss
    from sentence_transformers import SentenceTransformer
except:
    !pip install -q sentence-transformers faiss-cpu
    import faiss
    from sentence_transformers import SentenceTransformer

import numpy as np
import json
import os

# 2. CHEMIN VERS LES FICHIERS SAUVEGARDÉS
SAVE_DIR = "medical_rag_saved"  # Changez ce chemin si nécessaire

print(f"📁 Chargement depuis: {SAVE_DIR}")

# 3. CHARGER L'INDEX FAISS
print("🔍 Chargement de l'index FAISS...")
index = faiss.read_index(f"{SAVE_DIR}/medical_index.bin")
print(f"✅ Index chargé: {index.ntotal} documents")

# 4. CHARGER LES EMBEDDINGS
print("🧠 Chargement des embeddings...")
embeddings = np.load(f"{SAVE_DIR}/embeddings.npy")
print(f"✅ Embeddings chargés: {embeddings.shape}")

# 5. CHARGER LES CHUNKS
print("📄 Chargement des chunks...")
with open(f"{SAVE_DIR}/chunks.json", 'r', encoding='utf-8') as f:
    chunks = json.load(f)
print(f"✅ {len(chunks)} chunks chargés")

# 6. CHARGER LES DOCUMENTS
print("📚 Chargement des documents...")
with open(f"{SAVE_DIR}/documents.json", 'r', encoding='utf-8') as f:
    medical_documents = json.load(f)
print(f"✅ {len(medical_documents)} documents chargés")

# 7. CHARGER LES INFORMATIONS SYSTÈME
print("⚙️ Chargement des informations système...")
with open(f"{SAVE_DIR}/system_info.json", 'r', encoding='utf-8') as f:
    system_info = json.load(f)
print(f"✅ Système version {system_info.get('version', '1.0')} chargé")

# 8. CHARGER LE MODÈLE D'EMBEDDING
print("🤖 Chargement du modèle d'embedding...")
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
print("✅ Modèle chargé")

# 9. CHARGER LA CLASSE MedicalRAGSystem
print("🔄 Importation de la classe MedicalRAGSystem...")

# Importer la classe depuis le fichier sauvegardé
import sys
sys.path.append(SAVE_DIR)

from medical_rag_class import MedicalRAGSystem

# 10. INITIALISER LE SYSTÈME
print("🚀 Initialisation du système RAG...")
rag_system = MedicalRAGSystem(index, chunks, model)
print("✅ Système RAG prêt à l'emploi!")

# 11. TEST RAPIDE
print("\\n" + "="*60)
print("🧪 TEST RAPIDE DU SYSTÈME CHARGÉ")
print("="*60)

test_question = "Les antibiotiques fonctionnent-ils contre la grippe?"
print(f"\\n❓ Question test: {test_question}")

result = rag_system.generate_answer(test_question)
print(f"\\n🤖 Réponse:")
print(result["answer"])

if result["sources"]:
    print(f"\\n📚 Source(s) utilisée(s):")
    for i, source in enumerate(result["sources"], 1):
        print(f"  {i}. {source['title']} ({source['similarity']:.0%} pertinente)")

print(f"\\n🎯 Confiance: {result['confidence']:.0%}")

# 12. FONCTION UTILITAIRE POUR POSER DES QUESTIONS
def ask_medical_question(question):
    """Pose une question au système RAG médical"""
    result = rag_system.generate_answer(question)

    print(f"\\n{'='*60}")
    print(f"❓ Question: {question}")
    print(f"{'='*60}")
    print(f"\\n🤖 Réponse ({result['confidence']:.0%}):")
    print(result["answer"])

    if result["sources"]:
        print(f"\\n📖 Basé sur {len(result['sources'])} source(s):")
        for source in result["sources"]:
            print(f"  • {source['title']}")

    print(f"{'='*60}")
    return result

print("\\n" + "="*60)
print("💡 UTILISATION:")
print("="*60)
print("""
Pour poser une question, utilisez:
    result = rag_system.generate_answer("Votre question ici")

Ou utilisez la fonction utilitaire:
    ask_medical_question("Les vaccins COVID sont-ils sûrs?")

Le système est maintenant prêt à répondre à vos questions médicales!
""")

print("\\n" + "="*60)
print("✅ SYSTÈME RAG CHARGÉ AVEC SUCCÈS!")
print("="*60)
'''

with open(f"{save_dir}/load_medical_rag.py", 'w', encoding='utf-8') as f:
    f.write(loader_file_content)
print("✅ Fichier de chargement créé")

# 4. CRÉER UN NOTEBOOK DE DÉMO
notebook_content = '''
# NOTEBOOK DE DÉMONSTRATION - SYSTÈME RAG MÉDICAL
# ===============================================

## 🏥 Système de Question-Réponse Médical

Ce notebook démontre un système RAG (Retrieval-Augmented Generation) médical
qui répond à des questions de santé basées sur une base de connaissances.

### 📚 Base de connaissances incluse:
1. Antibiotiques et infections
2. Vaccins COVID-19
3. Prévention des infections
4. Symptômes COVID-19
5. Renforcement immunitaire
6. Traitement de la fièvre

### 🚀 Utilisation rapide:

```python
# Charger le système
from load_medical_rag import rag_system

# Poser une question
result = rag_system.generate_answer("Les antibiotiques soignent-ils la grippe?")
print(result["answer"])

# Utiliser la fonction utilitaire
from load_medical_rag import ask_medical_question
ask_medical_question("Comment prévenir le COVID?")